Test model data too see false positives

In [7]:
%matplotlib inline
from ultralytics import YOLO
from glob import glob
import matplotlib.pyplot as plt
from skimage import io
import numpy as np
image_reg = r'E:\Scott\smartLLSM\YOLO_demo\YOLO_demo\*.tif'
model_path = r'E:\Scott\GitHub\CelFDrive\Models\20240501_ScottlabelsV9\weights\best.pt'
model = YOLO(model_path)
image_files = glob(image_reg)

def preprocess_image(img):

    percentile = 0.01
    percentile_threshold = 100 - percentile  # 100 - 0.3
    threshold_value = np.percentile(img, percentile_threshold)
    # we assume 2D here so if its 3D at this point the 3rd index we assume is channels that are identical as others for saving
    if len(img.shape) == 3:
        h, w, _ = img.shape
        timepoint_data = img[:,:, 0]
    elif len(img.shape) == 2:
        h, w = img.shape
        timepoint_data = img
    else:
        print("Shape Error in predict.preprocess_image")



#     print(img.shape)
#     print(threshold_value)


    timepoint_data_normalized = np.where(timepoint_data > threshold_value, threshold_value, timepoint_data)

    # Normalize the image to the range of 0 to 1

    timepoint_data_normalized2 = (timepoint_data_normalized - timepoint_data_normalized.min()) / (
        timepoint_data_normalized.max() - timepoint_data_normalized.min())
    # plt.imshow(timepoint_data_normalized2)

    img = (timepoint_data_normalized2 * 255).astype(np.uint8)

    return np.repeat(img[:, :, np.newaxis], 3, axis=2)


for i, image_path in enumerate(image_files):
    # try:
    img = io.imread(image_path)
    print(f"Displaying {image_path} with shape {img.shape}")

    img = preprocess_image(img)

    results = model(img, conf =0.02)  # return a list of Results objects
    # # Process results list
    for result in results:
        boxes = result.boxes  # Boxes object for bounding box outputs
        print(boxes.shape[0])
        
        masks = result.masks  # Masks object for segmentation masks outputs
        keypoints = result.keypoints  # Keypoints object for pose outputs
        probs = result.probs  # Probs object for classification outputs
        obb = result.obb  # Oriented boxes object for OBB outputs
        if boxes.shape[0] > 0:
            result.save(filename=f'results{i}.jpg')



Displaying E:\Scott\smartLLSM\YOLO_demo\YOLO_demo\Scan_Iter_0000_0000.tif with shape (800, 800)

0: 640x640 (no detections), 150.9ms
Speed: 7.0ms preprocess, 150.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
0
Displaying E:\Scott\smartLLSM\YOLO_demo\YOLO_demo\Scan_Iter_0000_0001.tif with shape (800, 800)

0: 640x640 (no detections), 143.9ms
Speed: 6.0ms preprocess, 143.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
0
Displaying E:\Scott\smartLLSM\YOLO_demo\YOLO_demo\Scan_Iter_0000_0002.tif with shape (800, 800)

0: 640x640 (no detections), 139.9ms
Speed: 6.0ms preprocess, 139.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
0
Displaying E:\Scott\smartLLSM\YOLO_demo\YOLO_demo\Scan_Iter_0000_0003.tif with shape (800, 800)

0: 640x640 (no detections), 139.9ms
Speed: 6.0ms preprocess, 139.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
0
Displaying E:\Scott\smartLLSM\YOLO_demo\YOLO_demo\Scan_Iter_0000_000